In [1]:
import pandas as pd
import numpy as np

Thanks to Kriti Mahajan. This table was developed with study type features extracted from the meta data.
Code found here: https://colab.research.google.com/drive/1lay-yWJ5ohN94lQLHYIxtWMuDWtCRGy0#scrollTo=N-IhAeEkcxDe

Strategy:
1. Combine newest CoY dataset, Kaggle dataset, PubMed dataset and other annotation group's dataset all into one annotation table.
2. Feature engineer via revising ngram list/reverse engineer
3. If desperate, get synthetic labels via synthetic augmentation of text data (trade out word for synonym).

### Trying direct merge with mf and kaggle label set via sha.

In [151]:
mf = pd.read_csv('metadata_features-2.csv')
mf

,Unnamed: 0,paper_id,sha,title,abstract,publish_time,authors,url,Statistical Methods,statistical_method_details,Study Type,study_type_details,results,coronavirus,fatality,sample_size,paper_focus,variables_controlled_for
0,0,zjufx4fo,b2897e1277f56641193a6db73825f707eed3e4c9,Sequence requirements for RNA strand transfer ...,Nidovirus subgenomic mRNAs contain a leader se...,2001-12-17,"Pasternak, Alexander O.; van den Born, Erwin; ...",http://europepmc.org/articles/pmc125340?pdf=re...,[],NaN,['mutagenesis'],Here we present results of a comprehensive co-...,Here we present results of a comprehensive co-...,NaN,NaN,NaN,NaN,NaN
1,1,ymceytj3,e3d0d482ebd9a8ba81c254cc433f314142e72174,"Crystal structure of murine sCEACAM1a[1,4]: a ...",CEACAM1 is a member of the carcinoembryonic an...,2002-05-01,"Tan, Kemin; Zelus, Bruce D.; Meijers, Rob; Liu...",http://europepmc.org/articles/pmc125375?pdf=re...,[],NaN,[],NaN,Here we report the crystal structure of solubl...,NaN,NaN,NaN,NaN,NaN
2,2,wzj2glte,00b1d99e70f779eb4ede50059db469c65e8c1469,Synthesis of a novel hepatitis C virus protein...,Hepatitis C virus (HCV) is an important human ...,2001-07-16,"Xu, Zhenming; Choi, Jinah; Yen, T.S.Benedict; ...",https://www.ncbi.nlm.nih.gov/pmc/articles/PMC1...,[],NaN,[],NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,2sfqsfm1,cf584e00f637cbd8f1bb35f3f09f5ed07b71aeb0,Structure of coronavirus main proteinase revea...,The key enzyme in coronavirus polyprotein proc...,2002-07-01,"Anand, Kanchan; Palm, Gottfried J.; Mesters, J...",http://europepmc.org/articles/pmc126080?pdf=re...,['model'],Molecular modelling and mutagenesis data impli...,['mutagenesis'],Molecular modelling and mutagenesis data impli...,The study illustrates that RNA viruses have ev...,NaN,NaN,NaN,NaN,NaN
4,4,i0zym7iq,dde02f11923815e6a16a31dd6298c46b109c5dfa,Discontinuous and non-discontinuous subgenomic...,"Arteri-, corona-, toro- and roniviruses are ev...",2002-12-01,"van Vliet, A.L.W.; Smits, S.L.; Rottier, P.J.M...",http://europepmc.org/articles/pmc136939?pdf=re...,[],NaN,[],NaN,We show that BEV mRNAs 3–5 lack a leader seque...,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59125,59125,pwq5gir9,20f185b11fa23da8938efb5c839c6c078c66f1e9,Cutting-Edge Infectious Disease Diagnostics wi...,"Three recent Science articles (Chen et al., 20...",2018-06-13,"Chiu, Charles",https://doi.org/10.1016/j.chom.2018.05.016,[],NaN,['clinical'],"Three recent Science articles (Chen et al., 20...",NaN,NaN,NaN,NaN,NaN,NaN
59126,59126,3racv2qe,e84532333db8f219f20ae0a8fe18130c0d010b04,Intranasal administration of oxytocin: Behavio...,Abstract The intranasal (IN-) administration o...,2013-09-30,"Veening, Jan G.; Olivier, Berend",https://doi.org/10.1016/j.neubiorev.2013.04.012,[],NaN,['review'],The present review explores our current knowle...,Only very recently in a microdialysis study in...,NaN,NaN,NaN,NaN,NaN
59127,59127,jngto9kv,31105078a2953217223699d09c6a80d0f5edfdf6,Infecciones virales graves en pacientes inmuno...,Resumen Los virus tienen un papel importante d...,2011-04-30,"Díaz, A.; Zaragoza, R.; Granada, R.; Salavert, M.",https://doi.org/10.1016/j.medin.2011.01.001,[],NaN,['review'],A review of the literature has been performed ...,NaN,NaN,"Furthermore, ventilator-associated pneumonia (...",NaN,NaN,NaN
59128,59128,td0776wj,b1f977d58a0eba25acdee5912ef71c372b0632de,"18 Omics, Bioinformatics, and Infectious Disea...",Bioinformatics and the ‘omics’ studies are rap...,2011-12-31,"Paszkiewicz, Konrad H.; Giezen, Mark van der",https://doi.org/10.1016/b978-0-12-384890-1.000...,"['sequencing', 'methods']","In addition, we provide an overview of methods...",['bioinformatics'],Bioinformatics and the ‘omics’ studies are rap...,NaN,NaN,NaN,NaN,NaN,NaN


In [152]:
list(mf)

['Unnamed: 0',
 'paper_id',
 'sha',
 'title',
 'abstract',
 'publish_time',
 'authors',
 'url',
 'Statistical Methods',
 'statistical_method_details',
 'Study Type',
 'study_type_details',
 'results',
 'coronavirus',
 'fatality',
 'sample_size',
 'paper_focus',
 'variables_controlled_for']

In [153]:
features = mf.drop(columns = ['Unnamed: 0',
 'title',
 'abstract',
 'publish_time',
 'authors',
 'url',
 'Statistical Methods',
 'statistical_method_details',
 'study_type_details',
 'results',
 'coronavirus',
 'fatality',
 'sample_size',
 'paper_focus',
 'variables_controlled_for'])
features

,paper_id,sha,Study Type
0,zjufx4fo,b2897e1277f56641193a6db73825f707eed3e4c9,['mutagenesis']
1,ymceytj3,e3d0d482ebd9a8ba81c254cc433f314142e72174,[]
2,wzj2glte,00b1d99e70f779eb4ede50059db469c65e8c1469,[]
3,2sfqsfm1,cf584e00f637cbd8f1bb35f3f09f5ed07b71aeb0,['mutagenesis']
4,i0zym7iq,dde02f11923815e6a16a31dd6298c46b109c5dfa,[]
...,...,...,...
59125,pwq5gir9,20f185b11fa23da8938efb5c839c6c078c66f1e9,['clinical']
59126,3racv2qe,e84532333db8f219f20ae0a8fe18130c0d010b04,['review']
59127,jngto9kv,31105078a2953217223699d09c6a80d0f5edfdf6,['review']
59128,td0776wj,b1f977d58a0eba25acdee5912ef71c372b0632de,['bioinformatics']


In [154]:
features['Study Type'].value_counts()

[]                                                                           37266
['clinical']                                                                  5782
['review']                                                                    3594
['genetic']                                                                   3089
['review', 'clinical']                                                        1347
                                                                             ...  
['retrospective study', 'retrospectively analyzed']                              1
['retrospective study', 'histories']                                             1
['review', 'retrospective study', 'retrospectively analyzed', 'clinical']        1
['review', 'single center study', 'clinical']                                    1
['meta-analysis', 'clinical', 'bioinformatics']                                  1
Name: Study Type, Length: 393, dtype: int64

In [155]:
to_drop = ['[]']
features = features[~features['Study Type'].isin(to_drop)]
features

,paper_id,sha,Study Type
0,zjufx4fo,b2897e1277f56641193a6db73825f707eed3e4c9,['mutagenesis']
3,2sfqsfm1,cf584e00f637cbd8f1bb35f3f09f5ed07b71aeb0,['mutagenesis']
6,gi6uaa83,8ae137c8da1607b3a8e4c946c07ca8bda67f88ac,"['genetic', 'histories']"
7,0m32ecnu,23bc55d6f63fab18b02004483888db2b6a0bfa48,['genetic']
10,fy4w7xz8,0104f6ceccf92ae8567a0102f89cbb976969a774,['genetic']
...,...,...,...
59125,pwq5gir9,20f185b11fa23da8938efb5c839c6c078c66f1e9,['clinical']
59126,3racv2qe,e84532333db8f219f20ae0a8fe18130c0d010b04,['review']
59127,jngto9kv,31105078a2953217223699d09c6a80d0f5edfdf6,['review']
59128,td0776wj,b1f977d58a0eba25acdee5912ef71c372b0632de,['bioinformatics']


In [156]:
features['Study Type'].value_counts()

['clinical']                                                        5782
['review']                                                          3594
['genetic']                                                         3089
['review', 'clinical']                                              1347
['experimental']                                                    1241
                                                                    ... 
['review', 'retrospective cohort', 'case series', 'clinical']          1
['review', 'retrospective study', 'clinical', 'laboratory data']       1
['experimental', 'meta-analysis']                                      1
['time series', 'time-series', 'clinical']                             1
['mutagenesis', 'review', 'life cycle']                                1
Name: Study Type, Length: 392, dtype: int64

In [157]:
kaggle_labels = pd.read_csv('design-2.csv')

In [181]:
kg_merge = features.merge(kaggle_labels, how='inner', left_on='sha', right_on='id')
kg_merge

,paper_id,sha,Study Type,label,id
0,47ema2dq,6a3fa8ed278df0d05c5e009521de11c72308f60b,['review'],1,6a3fa8ed278df0d05c5e009521de11c72308f60b
1,9trzqzep,eb4b8f43f68db5e493984b7e4e56d9f804d7e885,"['questionnaire', 'cross-sectional']",8,eb4b8f43f68db5e493984b7e4e56d9f804d7e885
2,hnu8gw6w,2afcf2ab2330cb56ec493f82d87fa0c5b5f076e9,"['time series', 'clinical']",5,2afcf2ab2330cb56ec493f82d87fa0c5b5f076e9
3,5ulk3euw,9f7ed2037968c63fa97eb6fc7c35e9738a654541,['cross-sectional'],7,9f7ed2037968c63fa97eb6fc7c35e9738a654541
4,loowpch7,de707f97b534ca340482600ed1f008b20d7b1b8e,"['prospective cohort', 'prospective cohort']",4,de707f97b534ca340482600ed1f008b20d7b1b8e
...,...,...,...,...,...
493,dycx8i7h,2d0d272a793b99ef896547024c51f486e9552e5a,['genetic'],0,2d0d272a793b99ef896547024c51f486e9552e5a
494,p8vhi6d9,f778c6317cdc3e2c4aa7a8c7734fa13da10723eb,['experimental'],2,f778c6317cdc3e2c4aa7a8c7734fa13da10723eb
495,a6i73v0o,95d9bac431797e2cf8b7294a648ceec3bd50c84b,"['questionnaire', 'survey', 'cross-sectional']",7,95d9bac431797e2cf8b7294a648ceec3bd50c84b
496,i2vg8zqc,e9dfbc4084497e74dcaf6bd9c1dbde2b0fb70802,"['experimental', 'clinical']",0,e9dfbc4084497e74dcaf6bd9c1dbde2b0fb70802


Complete kg_merge annotation set by:
1. mapping labels to cw_labels
2. performing one-hot-encoding to output multi-label dataset

In [182]:
cw_labels = pd.read_csv('CoY_newlist.csv')

In [183]:
list(cw_labels)

['Assignee',
 'cord_uid',
 'sha',
 'title',
 'url',
 'Computational ',
 'Experimental-in vitro',
 'Experimental-in vivo',
 'Systematic review and/or meta-analysis',
 'Clinical-interventional',
 'Clinical-interventional.1',
 'Clinical-observational',
 'Clinical-observational.1',
 'Systematic review and/or meta-analysis ',
 'Other',
 'General Notes']

In [184]:
cw_labels = cw_labels.drop(columns = ['Other', 'General Notes'])

In [185]:
#Rename/Clean-up labels
cw_labels = cw_labels.rename(columns={'Computational ': 'Computational', 'Experimental-in vitro' : 'Experimental - in vitro', 'Experimental-in vivo' : 'Experimental - in vivo', 'Clinical-interventional.1' : 'Clinical-interventional', 'Clinical-observational.1' : 'Clinical-observational', 'Systematic review and/or meta-analysis ' : 'Systematic review and/or meta-analysis'})

In [186]:
list(cw_labels)

['Assignee',
 'cord_uid',
 'sha',
 'title',
 'url',
 'Computational',
 'Experimental - in vitro',
 'Experimental - in vivo',
 'Systematic review and/or meta-analysis',
 'Clinical-interventional',
 'Clinical-interventional',
 'Clinical-observational',
 'Clinical-observational',
 'Systematic review and/or meta-analysis']

Mapping Kaggle labels to CoronaWhy labels

0. Other -> Drop
1. Systematic review -> Systematic review and/or meta-analysis
2. Randomized control trial -> Clinical-interventional
3. Non-randomized trial -> Clinical - interventional
4. Prospective observational -> Clinical -observational
5. Time-to-event analysis -> Clinical-observational
6. Retrospective observational -> Clinical-observational
7. Cross-sectional -> Clinical-observational
8. Case series -> Clinical-observational
9. Modeling -> Computational

In [187]:
kg_merge.explode('label')
kg_merge['label'].unique()

array([1, 8, 5, 7, 4, 2, 9, 6, 0, 3])

In [188]:
dic_map={
         1:'Systematic review and/or meta-analysis',
         2:'Clinical-interventional',
         3:'Clinical-interventional',
         4:'Clinical-observational',
         5:'Clinical-observational',
         6:'Clinical-observational',
         7:'Clinical-observational',
         8:'Clinical-observational',
         9:'Computational',
         0:'Other'
        }

In [189]:
kg_merge['mapped'] = kg_merge.apply(lambda x: dic_map[x.label], axis = 1) 
kg_merge.head()

,paper_id,sha,Study Type,label,id,mapped
0,47ema2dq,6a3fa8ed278df0d05c5e009521de11c72308f60b,['review'],1,6a3fa8ed278df0d05c5e009521de11c72308f60b,Systematic review and/or meta-analysis
1,9trzqzep,eb4b8f43f68db5e493984b7e4e56d9f804d7e885,"['questionnaire', 'cross-sectional']",8,eb4b8f43f68db5e493984b7e4e56d9f804d7e885,Clinical-observational
2,hnu8gw6w,2afcf2ab2330cb56ec493f82d87fa0c5b5f076e9,"['time series', 'clinical']",5,2afcf2ab2330cb56ec493f82d87fa0c5b5f076e9,Clinical-observational
3,5ulk3euw,9f7ed2037968c63fa97eb6fc7c35e9738a654541,['cross-sectional'],7,9f7ed2037968c63fa97eb6fc7c35e9738a654541,Clinical-observational
4,loowpch7,de707f97b534ca340482600ed1f008b20d7b1b8e,"['prospective cohort', 'prospective cohort']",4,de707f97b534ca340482600ed1f008b20d7b1b8e,Clinical-observational


In [190]:
#Drop 0. Other label
kg_merge = kg_merge[kg_merge.label != 0]
kg_merge

,paper_id,sha,Study Type,label,id,mapped
0,47ema2dq,6a3fa8ed278df0d05c5e009521de11c72308f60b,['review'],1,6a3fa8ed278df0d05c5e009521de11c72308f60b,Systematic review and/or meta-analysis
1,9trzqzep,eb4b8f43f68db5e493984b7e4e56d9f804d7e885,"['questionnaire', 'cross-sectional']",8,eb4b8f43f68db5e493984b7e4e56d9f804d7e885,Clinical-observational
2,hnu8gw6w,2afcf2ab2330cb56ec493f82d87fa0c5b5f076e9,"['time series', 'clinical']",5,2afcf2ab2330cb56ec493f82d87fa0c5b5f076e9,Clinical-observational
3,5ulk3euw,9f7ed2037968c63fa97eb6fc7c35e9738a654541,['cross-sectional'],7,9f7ed2037968c63fa97eb6fc7c35e9738a654541,Clinical-observational
4,loowpch7,de707f97b534ca340482600ed1f008b20d7b1b8e,"['prospective cohort', 'prospective cohort']",4,de707f97b534ca340482600ed1f008b20d7b1b8e,Clinical-observational
...,...,...,...,...,...,...
490,l4649du3,bb3bf36ce2af876dcbf80b6cf3598e4421c98b3e,['clinical'],2,bb3bf36ce2af876dcbf80b6cf3598e4421c98b3e,Clinical-interventional
491,3vpoxfnv,eb720970f362e4416b73393f230c8606c837c6dc,['clinical'],9,eb720970f362e4416b73393f230c8606c837c6dc,Computational
494,p8vhi6d9,f778c6317cdc3e2c4aa7a8c7734fa13da10723eb,['experimental'],2,f778c6317cdc3e2c4aa7a8c7734fa13da10723eb,Clinical-interventional
495,a6i73v0o,95d9bac431797e2cf8b7294a648ceec3bd50c84b,"['questionnaire', 'survey', 'cross-sectional']",7,95d9bac431797e2cf8b7294a648ceec3bd50c84b,Clinical-observational


100 less annotations is a huge drop. 

### One-Hot Encoding

In [191]:
one_hot = pd.get_dummies(kg_merge['mapped'])
#drop mapped column as it is encoded now
kg_merge = kg_merge.drop('mapped',axis = 1)
# Join the encoded mapped to df3
kg_merge = kg_merge.join(one_hot)
kg_merge

,paper_id,sha,Study Type,label,id,Clinical-interventional,Clinical-observational,Computational,Systematic review and/or meta-analysis
0,47ema2dq,6a3fa8ed278df0d05c5e009521de11c72308f60b,['review'],1,6a3fa8ed278df0d05c5e009521de11c72308f60b,0,0,0,1
1,9trzqzep,eb4b8f43f68db5e493984b7e4e56d9f804d7e885,"['questionnaire', 'cross-sectional']",8,eb4b8f43f68db5e493984b7e4e56d9f804d7e885,0,1,0,0
2,hnu8gw6w,2afcf2ab2330cb56ec493f82d87fa0c5b5f076e9,"['time series', 'clinical']",5,2afcf2ab2330cb56ec493f82d87fa0c5b5f076e9,0,1,0,0
3,5ulk3euw,9f7ed2037968c63fa97eb6fc7c35e9738a654541,['cross-sectional'],7,9f7ed2037968c63fa97eb6fc7c35e9738a654541,0,1,0,0
4,loowpch7,de707f97b534ca340482600ed1f008b20d7b1b8e,"['prospective cohort', 'prospective cohort']",4,de707f97b534ca340482600ed1f008b20d7b1b8e,0,1,0,0
...,...,...,...,...,...,...,...,...,...
490,l4649du3,bb3bf36ce2af876dcbf80b6cf3598e4421c98b3e,['clinical'],2,bb3bf36ce2af876dcbf80b6cf3598e4421c98b3e,1,0,0,0
491,3vpoxfnv,eb720970f362e4416b73393f230c8606c837c6dc,['clinical'],9,eb720970f362e4416b73393f230c8606c837c6dc,0,0,1,0
494,p8vhi6d9,f778c6317cdc3e2c4aa7a8c7734fa13da10723eb,['experimental'],2,f778c6317cdc3e2c4aa7a8c7734fa13da10723eb,1,0,0,0
495,a6i73v0o,95d9bac431797e2cf8b7294a648ceec3bd50c84b,"['questionnaire', 'survey', 'cross-sectional']",7,95d9bac431797e2cf8b7294a648ceec3bd50c84b,0,1,0,0


### Clean up table by dropping unnecessary columns and adding necessary columns to kg_merge to final output.

In [192]:
kg_merge = kg_merge.drop(columns = ['label', 'id'])
kg_merge

,paper_id,sha,Study Type,Clinical-interventional,Clinical-observational,Computational,Systematic review and/or meta-analysis
0,47ema2dq,6a3fa8ed278df0d05c5e009521de11c72308f60b,['review'],0,0,0,1
1,9trzqzep,eb4b8f43f68db5e493984b7e4e56d9f804d7e885,"['questionnaire', 'cross-sectional']",0,1,0,0
2,hnu8gw6w,2afcf2ab2330cb56ec493f82d87fa0c5b5f076e9,"['time series', 'clinical']",0,1,0,0
3,5ulk3euw,9f7ed2037968c63fa97eb6fc7c35e9738a654541,['cross-sectional'],0,1,0,0
4,loowpch7,de707f97b534ca340482600ed1f008b20d7b1b8e,"['prospective cohort', 'prospective cohort']",0,1,0,0
...,...,...,...,...,...,...,...
490,l4649du3,bb3bf36ce2af876dcbf80b6cf3598e4421c98b3e,['clinical'],1,0,0,0
491,3vpoxfnv,eb720970f362e4416b73393f230c8606c837c6dc,['clinical'],0,0,1,0
494,p8vhi6d9,f778c6317cdc3e2c4aa7a8c7734fa13da10723eb,['experimental'],1,0,0,0
495,a6i73v0o,95d9bac431797e2cf8b7294a648ceec3bd50c84b,"['questionnaire', 'survey', 'cross-sectional']",0,1,0,0


In [193]:
#Adding necessary columns
c = {'Experimental - in vitro': 0, 'Experimental - in vivo': 0}
kg_merge = kg_merge.assign(**c)
#Reordering columns
cols = ['paper_id', 'sha', 'Study Type', 'Computational',
 'Experimental - in vitro',
 'Experimental - in vivo',
 'Clinical-interventional',
 'Clinical-observational',
 'Systematic review and/or meta-analysis']
kg_merge = kg_merge[cols]
kg_merge

,paper_id,sha,Study Type,Computational,Experimental - in vitro,Experimental - in vivo,Clinical-interventional,Clinical-observational,Systematic review and/or meta-analysis
0,47ema2dq,6a3fa8ed278df0d05c5e009521de11c72308f60b,['review'],0,0,0,0,0,1
1,9trzqzep,eb4b8f43f68db5e493984b7e4e56d9f804d7e885,"['questionnaire', 'cross-sectional']",0,0,0,0,1,0
2,hnu8gw6w,2afcf2ab2330cb56ec493f82d87fa0c5b5f076e9,"['time series', 'clinical']",0,0,0,0,1,0
3,5ulk3euw,9f7ed2037968c63fa97eb6fc7c35e9738a654541,['cross-sectional'],0,0,0,0,1,0
4,loowpch7,de707f97b534ca340482600ed1f008b20d7b1b8e,"['prospective cohort', 'prospective cohort']",0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...
490,l4649du3,bb3bf36ce2af876dcbf80b6cf3598e4421c98b3e,['clinical'],0,0,0,1,0,0
491,3vpoxfnv,eb720970f362e4416b73393f230c8606c837c6dc,['clinical'],1,0,0,0,0,0
494,p8vhi6d9,f778c6317cdc3e2c4aa7a8c7734fa13da10723eb,['experimental'],0,0,0,1,0,0
495,a6i73v0o,95d9bac431797e2cf8b7294a648ceec3bd50c84b,"['questionnaire', 'survey', 'cross-sectional']",0,0,0,0,1,0


### Onto cw_labels. 

1. Merge with features.
2. Combine like columns into 1 column.
2. Drop unncessary columns.
3. Reorder columns as in kg_merge.

In [207]:
cw_merge = features.merge(cw_labels, how='inner', left_on='paper_id', right_on='cord_uid')
cw_merge

,paper_id,sha_x,Study Type,Assignee,cord_uid,sha_y,title,url,Computational,Experimental - in vitro,Experimental - in vivo,Systematic review and/or meta-analysis,Clinical-interventional,Clinical-interventional,Clinical-observational,Clinical-observational,Systematic review and/or meta-analysis
0,lwla5ugt,8970ef10049816050cb1bc5fbf600865a5996ad4,['bioinformatics'],NaN,lwla5ugt,8970ef10049816050cb1bc5fbf600865a5996ad4,Base-By-Base: Single nucleotide-level analysis...,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC4...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,gdsfkw1b,8d7400a2b387820cd391d7df8194642e50402a0c,['review'],NaN,gdsfkw1b,8d7400a2b387820cd391d7df8194642e50402a0c,Protein secretion in Lactococcus lactis : an e...,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC5...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
2,e1iaiwc6,eddc547cbba693715e4fd55e6a946e8ec7d96bc2,['survey'],NaN,e1iaiwc6,eddc547cbba693715e4fd55e6a946e8ec7d96bc2,A Severe Acute Respiratory Syndrome extranet: ...,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC1...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,NaN
3,o088rxb5,9f63ebfaab049c0968d72166761217b3aaa1fe00,['clinical'],NaN,o088rxb5,9f63ebfaab049c0968d72166761217b3aaa1fe00,Pneumothorax and mortality in the mechanically...,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC1...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,NaN
4,icp9d998,8c640872048570b0a671ec08c83ef6cbebcdd78e,['clinical'],NaN,icp9d998,8c640872048570b0a671ec08c83ef6cbebcdd78e,Selection of a set of reliable reference genes...,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC1...,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
422,xk6tbnfu,23ead4b6087e730c7c24b626f490a75819eb038b,['clinical'],NaN,xk6tbnfu,23ead4b6087e730c7c24b626f490a75819eb038b,Bacterial Pneumonia in Older Adults,https://doi.org/10.1016/j.idc.2017.07.015,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
423,tbwecf7o,f60a18092c9596fed480a4e872150912bf3f9215,['prospective study'],NaN,tbwecf7o,f60a18092c9596fed480a4e872150912bf3f9215,Étude prospective de l’écologie virale hiverna...,https://doi.org/10.1016/j.arcped.2014.10.025,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
424,2lrq18w4,6448c25b3a8d2f818f50dbc0f3aa59d7d4ce32a1,['review'],NaN,2lrq18w4,6448c25b3a8d2f818f50dbc0f3aa59d7d4ce32a1,"Pediatric Ear, Nose, and Throat Emergencies",https://doi.org/10.1016/j.pcl.2005.10.002,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
425,jeivw926,6bef9eb1c377ae09eaaa5213fb68e74e9b6fe37d,"['review', 'genetic', 'clinical']",NaN,jeivw926,6bef9eb1c377ae09eaaa5213fb68e74e9b6fe37d,Perinatal death investigations: What is curren...,https://doi.org/10.1016/j.siny.2017.02.005,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0


In [210]:
cw_merge = cw_merge.groupby(cw_merge.columns, axis=1).sum()
cw_merge

,Assignee,Clinical-interventional,Clinical-observational,Computational,Experimental - in vitro,Experimental - in vivo,Study Type,Systematic review and/or meta-analysis,cord_uid,paper_id,sha_x,sha_y,title,url
0,0,0.0,0.0,0.0,0.0,0.0,['bioinformatics'],0.0,lwla5ugt,lwla5ugt,8970ef10049816050cb1bc5fbf600865a5996ad4,8970ef10049816050cb1bc5fbf600865a5996ad4,Base-By-Base: Single nucleotide-level analysis...,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC4...
1,0,0.0,0.0,0.0,0.0,0.0,['review'],1.0,gdsfkw1b,gdsfkw1b,8d7400a2b387820cd391d7df8194642e50402a0c,8d7400a2b387820cd391d7df8194642e50402a0c,Protein secretion in Lactococcus lactis : an e...,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC5...
2,0,0.0,0.0,0.0,0.0,0.0,['survey'],0.0,e1iaiwc6,e1iaiwc6,eddc547cbba693715e4fd55e6a946e8ec7d96bc2,eddc547cbba693715e4fd55e6a946e8ec7d96bc2,A Severe Acute Respiratory Syndrome extranet: ...,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC1...
3,0,0.0,0.0,0.0,0.0,0.0,['clinical'],0.0,o088rxb5,o088rxb5,9f63ebfaab049c0968d72166761217b3aaa1fe00,9f63ebfaab049c0968d72166761217b3aaa1fe00,Pneumothorax and mortality in the mechanically...,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC1...
4,0,0.0,0.0,0.0,1.0,0.0,['clinical'],0.0,icp9d998,icp9d998,8c640872048570b0a671ec08c83ef6cbebcdd78e,8c640872048570b0a671ec08c83ef6cbebcdd78e,Selection of a set of reliable reference genes...,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC1...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
422,0,0.0,0.0,0.0,0.0,0.0,['clinical'],0.0,xk6tbnfu,xk6tbnfu,23ead4b6087e730c7c24b626f490a75819eb038b,23ead4b6087e730c7c24b626f490a75819eb038b,Bacterial Pneumonia in Older Adults,https://doi.org/10.1016/j.idc.2017.07.015
423,0,0.0,0.0,0.0,0.0,0.0,['prospective study'],0.0,tbwecf7o,tbwecf7o,f60a18092c9596fed480a4e872150912bf3f9215,f60a18092c9596fed480a4e872150912bf3f9215,Étude prospective de l’écologie virale hiverna...,https://doi.org/10.1016/j.arcped.2014.10.025
424,0,0.0,0.0,0.0,0.0,0.0,['review'],0.0,2lrq18w4,2lrq18w4,6448c25b3a8d2f818f50dbc0f3aa59d7d4ce32a1,6448c25b3a8d2f818f50dbc0f3aa59d7d4ce32a1,"Pediatric Ear, Nose, and Throat Emergencies",https://doi.org/10.1016/j.pcl.2005.10.002
425,0,0.0,0.0,0.0,0.0,0.0,"['review', 'genetic', 'clinical']",1.0,jeivw926,jeivw926,6bef9eb1c377ae09eaaa5213fb68e74e9b6fe37d,6bef9eb1c377ae09eaaa5213fb68e74e9b6fe37d,Perinatal death investigations: What is curren...,https://doi.org/10.1016/j.siny.2017.02.005


In [213]:
cw_merge[cw_merge['Clinical-interventional'] > 1]

,Assignee,Clinical-interventional,Clinical-observational,Computational,Experimental - in vitro,Experimental - in vivo,Study Type,Systematic review and/or meta-analysis,cord_uid,paper_id,sha_x,sha_y,title,url


In [215]:
cw_merge[cw_merge['Clinical-observational'] > 1]

,Assignee,Clinical-interventional,Clinical-observational,Computational,Experimental - in vitro,Experimental - in vivo,Study Type,Systematic review and/or meta-analysis,cord_uid,paper_id,sha_x,sha_y,title,url


In [214]:
cw_merge[cw_merge['Computational'] > 1]

,Assignee,Clinical-interventional,Clinical-observational,Computational,Experimental - in vitro,Experimental - in vivo,Study Type,Systematic review and/or meta-analysis,cord_uid,paper_id,sha_x,sha_y,title,url


In [216]:
cw_merge[cw_merge['Experimental - in vitro'] > 1]

,Assignee,Clinical-interventional,Clinical-observational,Computational,Experimental - in vitro,Experimental - in vivo,Study Type,Systematic review and/or meta-analysis,cord_uid,paper_id,sha_x,sha_y,title,url


In [217]:
cw_merge[cw_merge['Experimental - in vivo'] > 1]

,Assignee,Clinical-interventional,Clinical-observational,Computational,Experimental - in vitro,Experimental - in vivo,Study Type,Systematic review and/or meta-analysis,cord_uid,paper_id,sha_x,sha_y,title,url


In [218]:
cw_merge[cw_merge['Systematic review and/or meta-analysis'] > 1]

,Assignee,Clinical-interventional,Clinical-observational,Computational,Experimental - in vitro,Experimental - in vivo,Study Type,Systematic review and/or meta-analysis,cord_uid,paper_id,sha_x,sha_y,title,url
289,0,0.0,0.0,0.0,0.0,0.0,"['review', 'clinical']",2.0,0cehgc34,0cehgc34,438f183a49a1c6cc580a3edea609bebeb2af0e9f,438f183a49a1c6cc580a3edea609bebeb2af0e9f,Efficacy and gastrointestinal risk of aspirin ...,https://doi.org/10.1016/j.bpg.2012.01.008
383,0,0.0,0.0,0.0,0.0,0.0,"['review', 'clinical']",2.0,hel3h25h,hel3h25h,4f3dfa1d4e3ffba1d199cbdf180baa47805a2e67,4f3dfa1d4e3ffba1d199cbdf180baa47805a2e67,Encephalitis diagnosis using metagenomics: app...,https://doi.org/10.1016/j.jinf.2017.12.014


Only column with values greater than 1 is 'Systematic review and/or meta-analysis'. Will change these values into 1.

In [220]:
list(kg_merge)

['paper_id',
 'sha',
 'Study Type',
 'Computational',
 'Experimental - in vitro',
 'Experimental - in vivo',
 'Clinical-interventional',
 'Clinical-observational',
 'Systematic review and/or meta-analysis']

In [221]:
list(cw_merge)

['Assignee',
 'Clinical-interventional',
 'Clinical-observational',
 'Computational',
 'Experimental - in vitro',
 'Experimental - in vivo',
 'Study Type',
 'Systematic review and/or meta-analysis',
 'cord_uid',
 'paper_id',
 'sha_x',
 'sha_y',
 'title',
 'url']

In [222]:
cw_merge = cw_merge.drop(columns = [
                      'Assignee',
                      'cord_uid',
                      'sha_y',
                      'title',
                      'url'])

,Clinical-interventional,Clinical-observational,Computational,Experimental - in vitro,Experimental - in vivo,Study Type,Systematic review and/or meta-analysis,paper_id,sha_x
0,0.0,0.0,0.0,0.0,0.0,['bioinformatics'],0.0,lwla5ugt,8970ef10049816050cb1bc5fbf600865a5996ad4
1,0.0,0.0,0.0,0.0,0.0,['review'],1.0,gdsfkw1b,8d7400a2b387820cd391d7df8194642e50402a0c
2,0.0,0.0,0.0,0.0,0.0,['survey'],0.0,e1iaiwc6,eddc547cbba693715e4fd55e6a946e8ec7d96bc2
3,0.0,0.0,0.0,0.0,0.0,['clinical'],0.0,o088rxb5,9f63ebfaab049c0968d72166761217b3aaa1fe00
4,0.0,0.0,0.0,1.0,0.0,['clinical'],0.0,icp9d998,8c640872048570b0a671ec08c83ef6cbebcdd78e
...,...,...,...,...,...,...,...,...,...
422,0.0,0.0,0.0,0.0,0.0,['clinical'],0.0,xk6tbnfu,23ead4b6087e730c7c24b626f490a75819eb038b
423,0.0,0.0,0.0,0.0,0.0,['prospective study'],0.0,tbwecf7o,f60a18092c9596fed480a4e872150912bf3f9215
424,0.0,0.0,0.0,0.0,0.0,['review'],0.0,2lrq18w4,6448c25b3a8d2f818f50dbc0f3aa59d7d4ce32a1
425,0.0,0.0,0.0,0.0,0.0,"['review', 'genetic', 'clinical']",1.0,jeivw926,6bef9eb1c377ae09eaaa5213fb68e74e9b6fe37d


In [223]:
cw_merge = cw_merge.rename(columns={'sha_x': 'sha'})
cw_merge

,Clinical-interventional,Clinical-observational,Computational,Experimental - in vitro,Experimental - in vivo,Study Type,Systematic review and/or meta-analysis,paper_id,sha
0,0.0,0.0,0.0,0.0,0.0,['bioinformatics'],0.0,lwla5ugt,8970ef10049816050cb1bc5fbf600865a5996ad4
1,0.0,0.0,0.0,0.0,0.0,['review'],1.0,gdsfkw1b,8d7400a2b387820cd391d7df8194642e50402a0c
2,0.0,0.0,0.0,0.0,0.0,['survey'],0.0,e1iaiwc6,eddc547cbba693715e4fd55e6a946e8ec7d96bc2
3,0.0,0.0,0.0,0.0,0.0,['clinical'],0.0,o088rxb5,9f63ebfaab049c0968d72166761217b3aaa1fe00
4,0.0,0.0,0.0,1.0,0.0,['clinical'],0.0,icp9d998,8c640872048570b0a671ec08c83ef6cbebcdd78e
...,...,...,...,...,...,...,...,...,...
422,0.0,0.0,0.0,0.0,0.0,['clinical'],0.0,xk6tbnfu,23ead4b6087e730c7c24b626f490a75819eb038b
423,0.0,0.0,0.0,0.0,0.0,['prospective study'],0.0,tbwecf7o,f60a18092c9596fed480a4e872150912bf3f9215
424,0.0,0.0,0.0,0.0,0.0,['review'],0.0,2lrq18w4,6448c25b3a8d2f818f50dbc0f3aa59d7d4ce32a1
425,0.0,0.0,0.0,0.0,0.0,"['review', 'genetic', 'clinical']",1.0,jeivw926,6bef9eb1c377ae09eaaa5213fb68e74e9b6fe37d


In [227]:
#Reorder columns
cols = list(kg_merge)
cw_merge = cw_merge[cols]
cw_merge

,paper_id,sha,Study Type,Computational,Experimental - in vitro,Experimental - in vivo,Clinical-interventional,Clinical-observational,Systematic review and/or meta-analysis
0,lwla5ugt,8970ef10049816050cb1bc5fbf600865a5996ad4,['bioinformatics'],0.0,0.0,0.0,0.0,0.0,0.0
1,gdsfkw1b,8d7400a2b387820cd391d7df8194642e50402a0c,['review'],0.0,0.0,0.0,0.0,0.0,1.0
2,e1iaiwc6,eddc547cbba693715e4fd55e6a946e8ec7d96bc2,['survey'],0.0,0.0,0.0,0.0,0.0,0.0
3,o088rxb5,9f63ebfaab049c0968d72166761217b3aaa1fe00,['clinical'],0.0,0.0,0.0,0.0,0.0,0.0
4,icp9d998,8c640872048570b0a671ec08c83ef6cbebcdd78e,['clinical'],0.0,1.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...
422,xk6tbnfu,23ead4b6087e730c7c24b626f490a75819eb038b,['clinical'],0.0,0.0,0.0,0.0,0.0,0.0
423,tbwecf7o,f60a18092c9596fed480a4e872150912bf3f9215,['prospective study'],0.0,0.0,0.0,0.0,0.0,0.0
424,2lrq18w4,6448c25b3a8d2f818f50dbc0f3aa59d7d4ce32a1,['review'],0.0,0.0,0.0,0.0,0.0,0.0
425,jeivw926,6bef9eb1c377ae09eaaa5213fb68e74e9b6fe37d,"['review', 'genetic', 'clinical']",0.0,0.0,0.0,0.0,0.0,1.0


In [230]:
columns = list(cw_merge.iloc[:,3:])
columns

['Computational',
 'Experimental - in vitro',
 'Experimental - in vivo',
 'Clinical-interventional',
 'Clinical-observational',
 'Systematic review and/or meta-analysis']

In [235]:
cw_merge[cw_merge['Systematic review and/or meta-analysis'] > 1]

,paper_id,sha,Study Type,Computational,Experimental - in vitro,Experimental - in vivo,Clinical-interventional,Clinical-observational,Systematic review and/or meta-analysis
289,0cehgc34,438f183a49a1c6cc580a3edea609bebeb2af0e9f,"['review', 'clinical']",0.0,0.0,0.0,0.0,0.0,2.0
383,hel3h25h,4f3dfa1d4e3ffba1d199cbdf180baa47805a2e67,"['review', 'clinical']",0.0,0.0,0.0,0.0,0.0,2.0


In [243]:
count = 0
for i in cw_merge.index:
    for c in columns:
        if (cw_merge[c][count] == 0) | (cw_merge[c][count] == 1.0):
            pass
        elif cw_merge[c][count] > 1:
            cw_merge.at[count,c] = 1
    count +=1

In [244]:
cw_merge[cw_merge['Systematic review and/or meta-analysis'] > 1]

,paper_id,sha,Study Type,Computational,Experimental - in vitro,Experimental - in vivo,Clinical-interventional,Clinical-observational,Systematic review and/or meta-analysis


Yes, all values in cw_merge labels are now between 0 and 1.

In [245]:
#Final check:
cw_merge

,paper_id,sha,Study Type,Computational,Experimental - in vitro,Experimental - in vivo,Clinical-interventional,Clinical-observational,Systematic review and/or meta-analysis
0,lwla5ugt,8970ef10049816050cb1bc5fbf600865a5996ad4,['bioinformatics'],0.0,0.0,0.0,0.0,0.0,0.0
1,gdsfkw1b,8d7400a2b387820cd391d7df8194642e50402a0c,['review'],0.0,0.0,0.0,0.0,0.0,1.0
2,e1iaiwc6,eddc547cbba693715e4fd55e6a946e8ec7d96bc2,['survey'],0.0,0.0,0.0,0.0,0.0,0.0
3,o088rxb5,9f63ebfaab049c0968d72166761217b3aaa1fe00,['clinical'],0.0,0.0,0.0,0.0,0.0,0.0
4,icp9d998,8c640872048570b0a671ec08c83ef6cbebcdd78e,['clinical'],0.0,1.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...
422,xk6tbnfu,23ead4b6087e730c7c24b626f490a75819eb038b,['clinical'],0.0,0.0,0.0,0.0,0.0,0.0
423,tbwecf7o,f60a18092c9596fed480a4e872150912bf3f9215,['prospective study'],0.0,0.0,0.0,0.0,0.0,0.0
424,2lrq18w4,6448c25b3a8d2f818f50dbc0f3aa59d7d4ce32a1,['review'],0.0,0.0,0.0,0.0,0.0,0.0
425,jeivw926,6bef9eb1c377ae09eaaa5213fb68e74e9b6fe37d,"['review', 'genetic', 'clinical']",0.0,0.0,0.0,0.0,0.0,1.0


Now, onto PubMed annotations. Need to:

1. Merge pm_labels to features
2. rename label columns to those corresponding to ones in kg_merge/cw_merge
3. Change False/True to 0/1
3. merge any same name columns and reconcile and values >1
3. Reorder column names

In [246]:
pm_labels = pd.read_csv('metadata_extra_info_from_pubmed_db.csv')
pm_labels

,is_rct,is_systematic_reivew,is_review,is_meta_analysis,is_observational_study,is_case_series,is_clincal_trial,cord_uid,pubmed_id
0,False,False,False,False,False,False,False,zjufx4fo,11742998
1,False,False,False,False,False,False,False,ymceytj3,11980704
2,False,False,False,False,False,False,False,wzj2glte,11447125
3,False,False,False,False,False,False,False,2sfqsfm1,12093723
4,False,False,False,False,False,False,False,i0zym7iq,12456663
...,...,...,...,...,...,...,...,...,...
57361,False,False,False,False,False,False,False,rn4gm3sc,32105468
57362,False,False,True,False,False,False,False,l5oecoot,16931122
57363,False,False,False,False,False,False,False,i4acg2ex,18035462
57364,False,False,True,False,False,False,False,rve0w3dg,8395326


In [248]:
pm_merge = features.merge(pm_labels, how='inner', left_on='paper_id', right_on='cord_uid')
pm_merge

,paper_id,sha,Study Type,is_rct,is_systematic_reivew,is_review,is_meta_analysis,is_observational_study,is_case_series,is_clincal_trial,cord_uid,pubmed_id
0,zjufx4fo,b2897e1277f56641193a6db73825f707eed3e4c9,['mutagenesis'],False,False,False,False,False,False,False,zjufx4fo,11742998
1,2sfqsfm1,cf584e00f637cbd8f1bb35f3f09f5ed07b71aeb0,['mutagenesis'],False,False,False,False,False,False,False,2sfqsfm1,12093723
2,gi6uaa83,8ae137c8da1607b3a8e4c946c07ca8bda67f88ac,"['genetic', 'histories']",False,False,True,False,False,False,False,gi6uaa83,12734001
3,0m32ecnu,23bc55d6f63fab18b02004483888db2b6a0bfa48,['genetic'],False,False,False,False,False,False,False,0m32ecnu,12881428
4,fy4w7xz8,0104f6ceccf92ae8567a0102f89cbb976969a774,['genetic'],False,False,False,False,False,False,False,fy4w7xz8,12969506
...,...,...,...,...,...,...,...,...,...,...,...,...
21238,pwq5gir9,20f185b11fa23da8938efb5c839c6c078c66f1e9,['clinical'],False,False,False,False,False,False,False,pwq5gir9,29902435
21239,3racv2qe,e84532333db8f219f20ae0a8fe18130c0d010b04,['review'],False,False,False,False,False,False,False,3racv2qe,23648680
21240,jngto9kv,31105078a2953217223699d09c6a80d0f5edfdf6,['review'],False,False,True,False,False,False,False,jngto9kv,21353339
21241,td0776wj,b1f977d58a0eba25acdee5912ef71c372b0632de,['bioinformatics'],False,False,False,False,False,False,False,td0776wj,0


Mapping pm_merge labels to cw_merge labels

1. 'is_rct' -> Clinical-interventional
2. 'is_systematic_reivew' -> Systematic review and/or meta-analysis
3. 'is_review' -> Other
4. 'is_meta_analysis' -> Systematic review and/or meta-analysis
5. 'is_observational_study' -> Clinical -observational
6. 'is_case_series' -> Clinical -observational
7. 'is_clincal_trial' -> Clinical -interventional

In [252]:
pm_merge = pm_merge.rename(columns=
{'is_rct': 'Clinical-interventional', 'is_systematic_reivew' : 'Systematic review and/or meta-analysis', 'is_review' : 'Other', 'is_meta_analysis' : 'Systematic review and/or meta-analysis', 'is_observational_study' : 'Clinical -observational',  'is_clincal_trial' : 'Clinical -interventional'})

In [253]:
pm_merge

,paper_id,sha,Study Type,Clinical-interventional,Systematic review and/or meta-analysis,Other,Systematic review and/or meta-analysis,Clinical -observational,is_case_series,Clinical -interventional,cord_uid,pubmed_id
0,zjufx4fo,b2897e1277f56641193a6db73825f707eed3e4c9,['mutagenesis'],False,False,False,False,False,False,False,zjufx4fo,11742998
1,2sfqsfm1,cf584e00f637cbd8f1bb35f3f09f5ed07b71aeb0,['mutagenesis'],False,False,False,False,False,False,False,2sfqsfm1,12093723
2,gi6uaa83,8ae137c8da1607b3a8e4c946c07ca8bda67f88ac,"['genetic', 'histories']",False,False,True,False,False,False,False,gi6uaa83,12734001
3,0m32ecnu,23bc55d6f63fab18b02004483888db2b6a0bfa48,['genetic'],False,False,False,False,False,False,False,0m32ecnu,12881428
4,fy4w7xz8,0104f6ceccf92ae8567a0102f89cbb976969a774,['genetic'],False,False,False,False,False,False,False,fy4w7xz8,12969506
...,...,...,...,...,...,...,...,...,...,...,...,...
21238,pwq5gir9,20f185b11fa23da8938efb5c839c6c078c66f1e9,['clinical'],False,False,False,False,False,False,False,pwq5gir9,29902435
21239,3racv2qe,e84532333db8f219f20ae0a8fe18130c0d010b04,['review'],False,False,False,False,False,False,False,3racv2qe,23648680
21240,jngto9kv,31105078a2953217223699d09c6a80d0f5edfdf6,['review'],False,False,True,False,False,False,False,jngto9kv,21353339
21241,td0776wj,b1f977d58a0eba25acdee5912ef71c372b0632de,['bioinformatics'],False,False,False,False,False,False,False,td0776wj,0


In [256]:
pm_merge = pm_merge.drop(columns = 'Other')

In [260]:
pm_merge = pm_merge.drop(columns = ['cord_uid', 'pubmed_id']) #when re-running notebook, merge this cell with prior one

In [261]:
pm_merge

,paper_id,sha,Study Type,Clinical-interventional,Systematic review and/or meta-analysis,Systematic review and/or meta-analysis,Clinical -observational,is_case_series,Clinical -interventional
0,zjufx4fo,b2897e1277f56641193a6db73825f707eed3e4c9,['mutagenesis'],False,False,False,False,False,False
1,2sfqsfm1,cf584e00f637cbd8f1bb35f3f09f5ed07b71aeb0,['mutagenesis'],False,False,False,False,False,False
2,gi6uaa83,8ae137c8da1607b3a8e4c946c07ca8bda67f88ac,"['genetic', 'histories']",False,False,False,False,False,False
3,0m32ecnu,23bc55d6f63fab18b02004483888db2b6a0bfa48,['genetic'],False,False,False,False,False,False
4,fy4w7xz8,0104f6ceccf92ae8567a0102f89cbb976969a774,['genetic'],False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...
21238,pwq5gir9,20f185b11fa23da8938efb5c839c6c078c66f1e9,['clinical'],False,False,False,False,False,False
21239,3racv2qe,e84532333db8f219f20ae0a8fe18130c0d010b04,['review'],False,False,False,False,False,False
21240,jngto9kv,31105078a2953217223699d09c6a80d0f5edfdf6,['review'],False,False,False,False,False,False
21241,td0776wj,b1f977d58a0eba25acdee5912ef71c372b0632de,['bioinformatics'],False,False,False,False,False,False


## Columns did not merge...

In [264]:
# pm_merge = pm_merge.groupby(pm_merge.columns, axis=1).sum()
# pm_merge